In [1]:
import pandas as pd

In [2]:
import pprint

In [3]:
import numpy as np

In [4]:
from IPython.core.display import HTML

In [5]:
df = pd.read_csv("agaricus-lepiota.data")

In [6]:
df.columns = df.columns.str.replace('.', '_')

/var/folders/nx/ty9f7d8x5gg5l6_b_xgdv8rc0000gn/T/ipykernel_9830/2262415099.py:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df.columns = df.columns.str.replace('.', '_')


In [7]:
for index, row in df.iterrows():
    for i in row:
        if i == "?":
            df.drop(index, inplace=True)
df = df.reset_index()

In [36]:
alter = df.iloc[:,1:]
alter

,p,x,s,n,t,p_1,f,c,n_1,k,...,s_2,w,w_1,p_2,w_2,o,p_3,k_1,s_3,u
0,e,x,s,y,t,a,f,c,b,k,...,s,w,w,p,w,o,p,n,n,g
1,e,b,s,w,t,l,f,c,b,n,...,s,w,w,p,w,o,p,n,n,m
2,p,x,y,w,t,p,f,c,n,n,...,s,w,w,p,w,o,p,k,s,u
3,e,x,s,g,f,n,f,w,b,k,...,s,w,w,p,w,o,e,n,a,g
4,e,x,y,y,t,a,f,c,b,n,...,s,w,w,p,w,o,p,k,n,g
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5638,e,b,y,n,f,n,f,c,b,w,...,y,n,n,p,w,t,p,w,y,p
5639,e,x,y,n,f,n,f,c,b,w,...,y,n,n,p,w,t,p,w,y,p
5640,e,x,y,g,t,n,f,c,b,w,...,s,w,w,p,w,t,p,w,y,p
5641,p,x,y,c,f,m,f,c,b,y,...,y,c,c,p,w,n,n,w,c,d


In [9]:
trainData = df.iloc[:,1:]
testSet = trainData.copy()

In [10]:
trainData = df.iloc[:-1000, 1:]
testSet = df.iloc[-1000:, 1:]
testSet = testSet.reset_index().iloc[:, 1:]

In [11]:
def calculateEnt(valueData, label, labelList):
    entropy = 0
    
    for l in labelList:
        labelCount = len(valueData[valueData[label] == l])
        if labelCount == 0:
            entropy += 0
        else:
            entropy += -(labelCount/valueData.size) * np.log2(labelCount/valueData.size)
    
    return entropy

In [12]:
def calculateIG(feature, data, label, labelList):
    valueList = data[feature].unique()
    currentEnt = 0
    rowCount = data.size
    expectedEnt = 0
    
    for l in labelList:
        count = len(data[data[label] == l])
        currentEnt += - (count/rowCount) * np.log2(count/rowCount)
    for value in valueList:
        valueData = data[data[feature] == value]
        count = valueData.size
        entropy = calculateEnt(valueData, label, labelList)
        expectedEnt += count/rowCount * entropy
    
    result = currentEnt - expectedEnt
    return result

In [13]:
def FindMaxIgFeature(data, label, labelList):
    maxIG = 0
    maxFeature = None
    
    featureData = data.columns.drop(label)
    
    for feature in featureData:
        IG = calculateIG(feature, data, label, labelList)
        if maxIG < IG:
            maxIG = IG
            maxFeature = feature
    return maxFeature

In [14]:
def generate(root, root_value, data, label, labelList, stopping_depth, depth):
    bottom = False
    if data.empty:
        return 0
    choice = None
    MaxIgFeature = FindMaxIgFeature(data, label, labelList)
    
    
    valueCount = data[MaxIgFeature].value_counts(sort=False)
    subtree = {}
    
    for featureValue, count in valueCount.iteritems():
        featureData = data[data[MaxIgFeature] == featureValue]
        
        isLeaf = False
        
        for l in labelList:
            labelCount = len(featureData[featureData[label] == l])
            
            if labelCount == count:
                isLeaf = True
                data = data[data[MaxIgFeature] != featureValue]
                subtree[featureValue] = l
        if not isLeaf:
            if stopping_depth == depth:
                maxCount = 0
                maxLabel = None
                for l in labelList:
                    labelCount = len(featureData[featureData[label] == l])
                    if labelCount > maxCount:
                        maxCount = labelCount
                        maxLabel = l
                subtree[featureValue] = maxLabel
            else:
                subtree[featureValue] = "?"
            # else:
            #     subtree[featureValue] = "terminate"
    
    if root_value:
        root[root_value] = {}
        root[root_value][MaxIgFeature] = subtree
        choice = root[root_value][MaxIgFeature]
    else:
        root[MaxIgFeature] = subtree
        choice = root[MaxIgFeature]

    for rule, output in choice.items():
        if output == "?":
            updated_data = data[data[MaxIgFeature] == rule]
            generate(choice, rule, updated_data, label, labelList, stopping_depth, depth + 1)

In [15]:
def predict(tree, row):
    if not isinstance(tree, dict):
        return tree
    else:
        rule = next(iter(tree))
        feature_value = row[rule]
        if feature_value in tree[rule]:
            return predict(tree[rule][feature_value], row)
        else:
            return None

In [29]:
def test(decisionTree, testSet, label):
    positive = 0
    negative = 0
    for index, row in testSet.iterrows():
        result = predict(decisionTree, testSet.iloc[index])
        if result == testSet[label].iloc[index]:
            positive += 1
        else:
            negative += 1
    return positive, negative, positive / (positive + negative)

In [30]:
def train(data, label, stopping_depth):
    decisionTree = {}
    generate(decisionTree, None, data, label, data[label].unique(), stopping_depth, 0)
    return decisionTree

In [31]:
def evaluate(traindata,testSet, label, maxDepth):
    li = []
    i = 0
    while i <= maxDepth:
        tree = train(trainData, label, i)
        if len(li) == 0:
            li.append(tree)
        elif tree != li[-1]:
            li.append(tree)
        else:
            break
        i += 1
    for j in li:
        print("depth " + str(li.index(j)))
        print(test(j, testSet, label))
        
    

In [32]:
evaluate(trainData,testSet, "p", 2)

depth 0
(879, 121, 0.879)
depth 1
(949, 51, 0.949)
depth 2
(912, 88, 0.912)


In [39]:
tree1 = train(alter, "p", 3)
pprint.pprint(tree1)

{'p_1': {'a': 'e',
         'c': 'p',
         'f': 'p',
         'l': 'e',
         'm': 'p',
         'n': {'k_1': {'k': 'e',
                       'n': 'e',
                       'r': 'p',
                       'w': {'n': {'c': 'e',
                                   'g': 'e',
                                   'n': 'e',
                                   'p': 'e',
                                   'w': 'p',
                                   'y': 'p'}}}},
         'p': 'p'}}


In [38]:
tree2 = train(trainData, "p", 3)
pprint.pprint(tree2)
print(test(tree2, testSet, "p"))

{'p_1': {'a': 'e',
         'c': 'p',
         'f': 'p',
         'l': 'e',
         'n': {'k_1': {'k': 'e',
                       'n': 'e',
                       'r': 'p',
                       'w': {'x': {'b': 'p', 'f': 'e', 'k': 'e'}}}},
         'p': 'p'}}
(912, 88, 0.912)
